In [5]:
import pandas as pd
import trackintel as ti
import gzip
import os
import geopandas as gpd

In [6]:
# Define the path to the .gz files
data_files = {
    'A': "/home/aden/Desktop/Data_mining/Part_2/13237029/13237029/cityA_groundtruthdata.csv",
    'B': "/home/aden/Desktop/Data_mining/Part_2/13237029/13237029/cityB_challengedata.csv", 
    'C': "/home/aden/Desktop/Data_mining/Part_2/13237029/13237029/cityC_challengedata.csv", 
    'D': "/home/aden/Desktop/Data_mining/Part_2/13237029/13237029/cityD_challengedata.csv" 
}


In [7]:
import pandas as pd
import trackintel as ti
import geopandas as gpd
from shapely import wkt
from joblib import Parallel, delayed
from itertools import combinations
from shapely.geometry import LineString
import re
import numpy as np


# Preprocessing: Load raw data, clean it, and save as a new CSV
def preprocess_data(city):
    """
    Preprocess the raw data for the specified city:
    1. Load the data.
    2. Remove invalid rows where x, y == -999.
    3. Add a 'tracked_at' timestamp column.
    4. Convert grid coordinates to approximate latitude/longitude.
    5. Save the processed data to a new CSV.
    """
    print(f"Loading raw data for city {city}...")
    df = pd.read_csv(data_files[city])

    # Drop rows with missing data (x, y marked as 999)
    print("Cleaning data: Removing rows with invalid coordinates...")
    df = df[(df['x'] != 999) & (df['y'] != 999)]

    # Add timestamp column
    print("Adding timestamp data...")
    df['date'] = pd.to_datetime(df['d'], format='%j', errors='coerce')
    df['time'] = pd.to_timedelta(df['t'] * 30, unit='m')
    df['tracked_at'] = df['date'] + df['time']
    df['tracked_at'] = df['tracked_at'].dt.tz_localize('UTC')

    # Convert grid coordinates to latitude/longitude
    print("Converting grid coordinates to latitude and longitude...")
    lat0 = 35.0  # Starting latitude
    lon0 = 135.0  # Starting longitude
    delta_degree = 0.0045  # Approx. 500m in degrees
    df['latitude'] = lat0 + (df['y'] - 1) * delta_degree
    df['longitude'] = lon0 + (df['x'] - 1) * delta_degree

    # Rename columns
    print("Renaming columns for consistency...")
    df.rename(columns={'uid': 'user_id'}, inplace=True)

    # Save the preprocessed data
    processed_file = f'data_{city}_preprocessed.csv'
    df.to_csv(processed_file, index=False)
    print(f"Data for city {city} saved to {processed_file}")

# Generate triplegs from preprocessed data
def gen_triplegs(city):
    """
    Generate triplegs for the specified city:
    1. Preprocess raw data.
    2. Generate staypoints using positionfixes.
    3. Generate triplegs between staypoints.
    4. Save triplegs to a CSV.
    """
    preprocess_data(city)

    print(f"Loading preprocessed data for city {city}...")
    # Load preprocessed data
    pfs = ti.io.file.read_positionfixes_csv(
        f'data_{city}_preprocessed.csv',
        sep=',',
        tz='utc',
        index_col=None,
        columns={
            'user_id': 'user_id',
            'tracked_at': 'tracked_at',
            'longitude': 'longitude',
            'latitude': 'latitude'
        },
        crs='EPSG:4326'
    )

    # Generate staypoints
    print("Generating staypoints from positionfixes...")
    pfs, sp = pfs.as_positionfixes.generate_staypoints(
        method='sliding',
        dist_threshold=0.5,  # Distance threshold (0.5 km)
        time_threshold=900,  # Time threshold (15 minutes)
        gap_threshold=300,
        distance_metric='haversine',
        include_last=True,
        print_progress=True,
        exclude_duplicate_pfs=True,
        n_jobs=-1
    )

    # Generate triplegs
    print("Generating triplegs from staypoints...")
    pfs, tpls = pfs.as_positionfixes.generate_triplegs(
        staypoints=sp, method='between_staypoints', gap_threshold=90
    )

    # Save triplegs
    print("Saving triplegs to CSV...")
    custom_write_triplegs_csv(tpls, f'triplegs_{city}.csv')

# Save triplegs with geometries converted to WKT
def custom_write_triplegs_csv(triplegs, filename):
    """
    Save triplegs to a CSV file, ensuring geometries are converted to WKT format.
    """
    print("Converting geometries to WKT format...")
    triplegs.loc[:, 'geom'] = triplegs['geom'].apply(
        lambda x: x.wkt if x and isinstance(x, LineString) else None
    )
    triplegs.to_csv(filename)
    print(f"Triplegs saved to {filename}")

# Split long triplegs into smaller segments
def split_long_triplegs(df, max_length=1000):
    """
    Split triplegs with a large number of points into smaller segments.
    """
    print("Splitting long triplegs into smaller segments...")
    new_rows = []
    for _, row in df.iterrows():
        geom = wkt.loads(row['geom'])
        coords = list(geom.coords)
        if len(coords) > max_length:
            for i in range(0, len(coords), max_length):
                new_geom = coords[i:i+max_length]
                new_linestring = LineString(new_geom)
                new_row = row.copy()  # Ensure new_row is an independent copy
                new_row['geom'] = wkt.dumps(new_linestring)
                new_rows.append(new_row)
        else:
            new_rows.append(row)
    print(f"Completed splitting triplegs. Total segments: {len(new_rows)}")
    return pd.DataFrame(new_rows)

def geom_to_grid_coords(geom):
    coords = np.array(geom.coords)
    # Convert latitude and longitude back to grid cell numbers
    delta_degree = 0.0045
    x_grid = ((coords[:, 0] - 135.0) / delta_degree + 1).astype(int)
    y_grid = ((coords[:, 1] - 35.0) / delta_degree + 1).astype(int)
    grid_coords = list(zip(x_grid, y_grid))  # Ensure this outputs tuples
    return grid_coords


def preprocess_triplegs(df, timestamp_col='started_at'):
    print("Preprocessing triplegs for sequential pattern mining...")
    df[timestamp_col] = pd.to_datetime(df[timestamp_col])

    # Filter data for the first 30 days
    print("Filtering triplegs to include only the first 30 days...")
    start_date = df[timestamp_col].min()
    end_date = start_date + pd.Timedelta(days=30)
    df = df[(df[timestamp_col] >= start_date) & (df[timestamp_col] < end_date)]

    # Parse 'geom' column into grid coordinates
    print("Parsing geometries into grid coordinates...")
    df.loc[:, 'geom'] = df['geom'].apply(wkt.loads)
    df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)

    # Group sequences by user
    user_sequences = df.groupby('user_id')['coords'].apply(list).tolist()

    # Flatten the sequences and ensure tuples
    sequences = []
    for user_seq in user_sequences:
        seq = []
        for tripleg_coords in user_seq:
            seq.extend(tripleg_coords)  # Extend the sequence
        sequences.append(tuple(seq))  # Convert entire sequence to a tuple
    return sequences


# Generalized Sequential Pattern (GSP) mining
def gsp(sequences, min_support):
    """
    Implement the GSP algorithm to find frequent sequential patterns.
    """
    print("Running GSP algorithm...")
    freq_seqs = {}  # Dictionary to store frequent sequences
    length = 1  # Length of sequences
    all_frequent_sequences = []  # List to store all frequent sequences

    # Generate 1-sequences
    print("Generating 1-sequences...")
    candidates = {}
    for sequence in sequences:
        for item in sequence:
            candidates[item] = candidates.get(item, 0) + 1
    freq_seqs[1] = { (item,): count for item, count in candidates.items() if count >= min_support }
    all_frequent_sequences.extend(freq_seqs[1].keys())

    # Generate longer sequences
    length = 2
    while freq_seqs.get(length - 1):
        print(f"Generating candidates for length {length}...")
        candidates = generate_candidates(freq_seqs[length - 1].keys(), length)
        support_count = count_support_parallel(candidates, sequences)
        freq_seqs[length] = prune_candidates(support_count, min_support)
        all_frequent_sequences.extend(freq_seqs[length].keys())
        print(f"Frequent sequences of length {length}: {len(freq_seqs[length])}")
        length += 1
    print("GSP algorithm completed.")
    return all_frequent_sequences




In [8]:
from tqdm.contrib.concurrent import process_map
from multiprocessing import Pool

def is_subsequence(candidate, sequence):
    """
    Efficiently check if 'candidate' is a subsequence of 'sequence'.
    """
    candidate_set = set(candidate)
    sequence_set = set(sequence)
    # Quick check: if candidate items aren't all in sequence, return False
    if not candidate_set.issubset(sequence_set):
        return False

    # Use iterators for efficient traversal
    seq_iter = iter(sequence)
    return all(item in seq_iter for item in candidate)

# Define 'single_candidate_support' at the top level
def single_candidate_support(args):
    """
    Worker function to count support for a single candidate.
    """
    candidate, sequences = args
    count = sum(is_subsequence(candidate, seq) for seq in sequences)
    return candidate, count

def count_support_parallel(candidates, sequences):
    """
    Count support for candidates using multiprocessing.Pool and tqdm for progress bar support.
    """
    if not candidates:
        print("No candidates to process!")
        return {}

    print(f"Counting support for {len(candidates)} candidates...")

    # Prepare arguments for worker processes
    args = [(candidate, sequences) for candidate in candidates]

    results = []
    with Pool() as pool:
        # Use tqdm to display progress
        for result in tqdm(pool.imap_unordered(single_candidate_support, args),
                           total=len(candidates), desc="Processing candidates"):
            results.append(result)

    print("Support counting finished.")
    return dict(results)


def generate_candidates(prev_freq_seqs, length):
    """
    Generate candidate sequences of the specified length.
    
    Parameters:
    - prev_freq_seqs: List of frequent sequences from the previous iteration.
    - length: Length of sequences to generate.

    Returns:
    - candidates: A set of candidate sequences of the specified length.
    """
    print(f"Generating candidates for sequences of length {length}...")
    candidates = set()
    prev_freq_seqs = list(prev_freq_seqs)
    for i in range(len(prev_freq_seqs)):
        for j in range(len(prev_freq_seqs)):
            seq1 = prev_freq_seqs[i]
            seq2 = prev_freq_seqs[j]
            # Check if the last n-1 elements of seq1 match the first n-1 elements of seq2
            if seq1[1:] == seq2[:-1]:
                candidate = seq1 + (seq2[-1],)
                candidates.add(candidate)
    return candidates

def filter_candidates_early(candidates, item_frequencies, min_support):
    print("Filtering candidates early...")
    filtered = [
        candidate for candidate in candidates
        if all(item_frequencies.get(item, 0) >= min_support for item in candidate)
    ]
    print(f"Filtered candidates from {len(candidates)} to {len(filtered)}")
    return filtered


from tqdm import tqdm

from itertools import islice


def prune_candidates(support_count, min_support):
    """
    Prune candidate sequences based on the minimum support threshold.

    Parameters:
    - support_count: Dictionary of candidate sequences and their support counts.
    - min_support: Minimum support threshold.

    Returns:
    - pruned_candidates: Dictionary of pruned candidates that meet the minimum support threshold.
    """
    return {seq: count for seq, count in support_count.items() if count >= min_support}

def is_subsequence(candidate, sequence):
    """
    Efficiently check if 'candidate' is a subsequence of 'sequence'.
    """
    seq_iter = iter(sequence)
    return all(item in seq_iter for item in candidate)

def save_gsp_results(gsp_results, output_file):
    """
    Save the GSP results to a CSV file.

    Parameters:
    - gsp_results: List of frequent sequences.
    - output_file: Path to the output file.
    """
    print(f"Saving GSP results to {output_file}...")
    sequences_df = pd.DataFrame(gsp_results, columns=['Sequence'])
    sequences_df.to_csv(output_file, index=False)
    print(f"GSP results saved to {output_file}")

In [10]:
# Main execution
if __name__ == "__main__":
    for i in ["A","B","C","D"]:
        # Read triplegs data
        df = pd.read_csv(f'triplegs_{i}.csv')

        # Split long triplegs
        df = split_long_triplegs(df)

        # Preprocess triplegs to get sequences
        sequences = preprocess_triplegs(df)

        # Set minimum support
        min_support = int(0.5 * len(sequences))  # Adjust as needed

        print(f"minium support {i}= {min_support}")

Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 13467512
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_2900/1398267354.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


minium support A= 49883
Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 3016752
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_2900/1398267354.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


minium support B= 12448
Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 2382997
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_2900/1398267354.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


minium support C= 9970
Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 632435
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_2900/1398267354.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


minium support D= 2991


In [9]:
# Main execution
if __name__ == "__main__":
    # Read triplegs data
    df = pd.read_csv('triplegs_B.csv')

    # Split long triplegs
    df = split_long_triplegs(df)

    # Preprocess triplegs to get sequences
    sequences = preprocess_triplegs(df)

    # Set minimum support
    min_support = int(0.5 * len(sequences))  # Adjust as needed

    # Run GSP algorithm
    frequent_sequences = gsp(sequences, min_support)

    # Save results
    save_gsp_results(frequent_sequences, 'frequent_sequences_B.csv')

Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 3016752
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_2900/1398267354.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


Running GSP algorithm...
Generating 1-sequences...
Generating candidates for length 2...
Generating candidates for sequences of length 2...
Counting support for 10404 candidates...


Processing candidates:   0%|          | 1/10404 [00:40<117:43:38, 40.74s/it]


KeyboardInterrupt: 

In [ ]:
# Main execution
if __name__ == "__main__":
    # Read triplegs data
    df = pd.read_csv('triplegs_A.csv')

    # Split long triplegs
    df = split_long_triplegs(df)

    # Preprocess triplegs to get sequences
    sequences = preprocess_triplegs(df)

    # Set minimum support
    min_support = int(0.5 * len(sequences))  # Adjust as needed

    # Run GSP algorithm
    frequent_sequences = gsp(sequences, min_support)

    # Save results
    save_gsp_results(frequent_sequences, 'frequent_sequences_A.csv')

Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 13467512
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_176184/1398267354.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


Running GSP algorithm...
Generating 1-sequences...
Generating candidates for length 2...
Generating candidates for sequences of length 2...
Counting support for 4 candidates...


Processing candidates: 100%|██████████| 4/4 [07:12<00:00, 108.23s/it]


Support counting finished.
Frequent sequences of length 2: 0
GSP algorithm completed.
Saving GSP results to frequent_sequences_A.csv...
GSP results saved to frequent_sequences_A.csv


In [ ]:
# Main execution
if __name__ == "__main__":
    # Read triplegs data
    df = pd.read_csv('triplegs_A.csv')

    # Split long triplegs
    df = split_long_triplegs(df)

    # Preprocess triplegs to get sequences
    sequences = preprocess_triplegs(df)

    # Set minimum support
    min_support = int(0.5 * len(sequences))  # Adjust as needed
    print(f"minium support = {min_support}")
    # Run GSP algorithm
    frequent_sequences = gsp(sequences, min_support)

    # Save results
    save_gsp_results(frequent_sequences, 'frequent_sequences_A.csv')

Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 13467512
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_190575/1398267354.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


minium support = 49883
Running GSP algorithm...
Generating 1-sequences...
Generating candidates for length 2...
Generating candidates for sequences of length 2...
Counting support for 64 candidates...


Processing candidates:   8%|▊         | 5/64 [09:04<1:45:22, 107.16s/it]

In [ ]:
# Main execution
if __name__ == "__main__":
    # Read triplegs data
    df = pd.read_csv('triplegs_C.csv')

    # Split long triplegs
    df = split_long_triplegs(df)

    # Preprocess triplegs to get sequences
    sequences = preprocess_triplegs(df)

    # Set minimum support
    min_support = int(0.8 * len(sequences))  # Adjust as needed

    # Run GSP algorithm
    frequent_sequences = gsp(sequences, min_support)

    # Save results
    save_gsp_results(frequent_sequences, 'frequent_sequences_C.csv')

In [ ]:
# Main execution
if __name__ == "__main__":
    # Read triplegs data
    df = pd.read_csv('triplegs_A.csv')

    # Split long triplegs
    df = split_long_triplegs(df)

    # Preprocess triplegs to get sequences
    sequences = preprocess_triplegs(df)

    # Set minimum support
    min_support = int(0.5 * len(sequences))  # Adjust as needed

    # Run GSP algorithm
    frequent_sequences = gsp(sequences, min_support)

    # Save results
    save_gsp_results(frequent_sequences, 'frequent_sequences_A.csv')

Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 13467512
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_560926/1380636941.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


Running GSP algorithm...
Generating 1-sequences...
Generating candidates for length 2...
Generating candidates for sequences of length 2...
Counting support for 64 candidates...


Processing candidates:   2%|▏         | 1/64 [02:05<2:11:21, 125.10s/it]

In [ ]:
# Main execution
if __name__ == "__main__":
    gen_triplegs("B")
    # Read triplegs data
    df = pd.read_csv('triplegs_B.csv')

    # Split long triplegs
    df = split_long_triplegs(df)

    # Preprocess triplegs to get sequences
    sequences = preprocess_triplegs(df)

    # Set minimum support
    min_support = int(0.8 * len(sequences))  # Adjust as needed

    # Run GSP algorithm
    frequent_sequences = gsp(sequences, min_support)

    # Save results
    save_gsp_results(frequent_sequences, 'frequent_sequences_B.csv')

Loading raw data for city B...
Cleaning data: Removing rows with invalid coordinates...
Adding timestamp data...
Converting grid coordinates to latitude and longitude...
Renaming columns for consistency...
Data for city B saved to data_B_preprocessed.csv
Loading preprocessed data for city B...
Generating staypoints from positionfixes...


100%|██████████| 25000/25000 [00:35<00:00, 709.21it/s] 


Generating triplegs from staypoints...


/home/aden/anaconda3/envs/data_mining/lib/python3.10/site-packages/trackintel/preprocessing/positionfixes.py:361: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pfs["tripleg_id"] = pfs["tripleg_id"].ffill()
/home/aden/anaconda3/envs/data_mining/lib/python3.10/site-packages/trackintel/preprocessing/positionfixes.py:573: UserWarning: The positionfixes with ids [     168      169      256 ... 23912055 23912826 23912827] lead to invalid tripleg geometries. The resulting triplegs were omitted and the tripleg id of the positionfixes was set to nan
  warnings.warn(warn_string)


Saving triplegs to CSV...
Converting geometries to WKT format...


/tmp/ipykernel_487307/1380636941.py:110: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['LINESTRING (135.3555 35.45, 135.3555 35.4455, 135.3555 35.45, 135.3555 35.45)'
 'LINESTRING (135.3645 35.4545, 135.4005 35.4815, 135.4005 35.4815, 135.4005 35.4815, 135.4005 35.4815, 135.4005 35.4815, 135.4005 35.4815, 135.4095 35.486, 135.4005 35.4815, 135.4005 35.4815, 135.4005 35.4815, 135.4005 35.486, 135.4005 35.4815, 135.4005 35.4815)'
 'LINESTRING (135.3465 35.4365, 135.342 35.4275, 135.3465 35.441, 135.3555 35.45)'
 ... 'LINESTRING (135.387 35.6705, 135.387 35.666)'
 'LINESTRING (135.3825 35.6795, 135.3645 35.675, 135.252 35.666, 135.225 35.648, 135.207 35.621, 135.207 35.621, 135.207 35.621, 135.2475 35.657, 135.3645 35.666, 135.387 35.666)'
 'LINESTRING (135.387 35.666, 135.387 35.666, 135.0585 35.0495, 135.2295 35.198, 135.3915 35.6255)']' has dtype incompatible with geometry, please explicitly cast to a compatible d

Triplegs saved to triplegs_B.csv
Splitting long triplegs into smaller segments...
Completed splitting triplegs. Total segments: 3016752
Preprocessing triplegs for sequential pattern mining...
Filtering triplegs to include only the first 30 days...
Parsing geometries into grid coordinates...


/tmp/ipykernel_487307/1380636941.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'coords'] = df['geom'].apply(geom_to_grid_coords)


Running GSP algorithm...
Generating 1-sequences...
Generating candidates for length 2...
Generating candidates for sequences of length 2...
Counting support for 961 candidates...


Processing candidates:   1%|▏         | 13/961 [05:09<6:13:18, 23.63s/it]